Here I preform classification at the wafer level. 
I transfer the data set to a format that works with this task, I do this by making each example x in X to be a 1D array of size without the WaferName and the corresponding y, if the wafer is defective or not.

Utility functions

In [19]:
# Here we will prepare the data for training by padding the wafers to be the same size, and 

def prepare_data_for_train(df_wafers):
    import numpy as np

    wafer_names = df_wafers['WaferName'].unique()
    wafer_groups = df_wafers.groupby('WaferName')
    df_wafers_split = [wafer_groups.get_group(wafer) for wafer in wafer_names]

    X = [df_wafers_split[i].drop(['WaferName','IsScratchDie'], axis=1) for i in range(len(df_wafers_split))]
    y = [df_wafers_split[i]['IsScratchDie'].any() for i in range(len(df_wafers_split))]


    #change the IsGoodDie column to 0 and 1 in X and the same for IsScratchDie in y
    y = [1 if i == True else 0 for i in y]
    X = [X[i].replace({'IsGoodDie': {True: 1, False: 0}}) for i in range(len(X))]

    #pudding
    wafer_size = [X[i].shape[0] for i in range(len(X))]

    #the wafers are not all the same size, so we need to pad the smaller ones with 0s rows

    max_size = max(wafer_size)

    X_padded = [X[i].append(pd.DataFrame(np.zeros((max_size - X[i].shape[0], X[i].shape[1])), columns=X[i].columns)) for i in range(len(X))]

    X_flat = [X_padded[i].values.flatten() for i in range(len(X_padded))]

    return X_flat, y


def prepare_data_for_test(df_wafers_test):
    import numpy as np

    wafer_names = df_wafers['WaferName'].unique()
    wafer_groups = df_wafers.groupby('WaferName')
    df_wafers_split = [wafer_groups.get_group(wafer) for wafer in wafer_names]

    X = [df_wafers_split[i].drop(['WaferName'], axis=1) for i in range(len(df_wafers_split))]


    #change the IsGoodDie column to 0 and 1 in X and the same for IsScratchDie in y
    X = [X[i].replace({'IsGoodDie': {True: 1, False: 0}}) for i in range(len(X))]

    #pudding
    wafer_size = [X[i].shape[0] for i in range(len(X))]

    #the wafers are not all the same size, so we need to pad the smaller ones with 0s rows

    max_size = max(wafer_size)

    X_padded = [X[i].append(pd.DataFrame(np.zeros((max_size - X[i].shape[0], X[i].shape[1])), columns=X[i].columns)) for i in range(len(X))]

    X_flat = [X_padded[i].values.flatten() for i in range(len(X_padded))]

    return X_flat

    
def train_model1(X, y):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)

    rf.fit(X_train, y_train)

    predictions = rf.predict(X_test)

    return rf, predictions, y_test


   
def plt_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title('Confusion matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def check_train_preds(predictions, y_test):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix

    print('Accuracy: ', accuracy_score(y_test, predictions))
    print('Confusion Matrix: ', confusion_matrix(y_test, predictions))


MAIN CODE

In [16]:
import pandas as pd
import zipfile

print('Loading data...')
#load zip file
zf = zipfile.ZipFile('data.zip') 
#load train data
df_wafers = pd.read_csv(zf.open('df_wafers.csv'))

#load test data
df_wafers_test = pd.read_csv(zf.open('df_wafers_test.csv'))

import warnings
warnings.filterwarnings('ignore')

print('Preparing data for training...')
X, y = prepare_data_for_train(df_wafers)

print('Training model...')
rf, predictions, y_test = train_model1(X, y)

check_train_preds(predictions, y_test)




Loading data...
Preparing data for training...
Training model...
Accuracy:  0.9228316326530612
Confusion Matrix:  [[747  30]
 [ 91 700]]
